<a href="https://colab.research.google.com/github/mgite03/bu-ai4all-2019/blob/main/rl/Copy_of_Intro_to_RL_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Review From Part 1

Define the following terms:

1. Agent

> The object in the environment that completes actions.

2. Environment

> The space where the agent performs actions.

3. State (and MDP)

> The location of the agent, and description of environment. States are independent from previous ones. 

4. Action

> What the agent does to change its environment/state. The agent recieves a reward for certain actions. 

5. Episode

> The collection of actions that the agent does, ends in reward or death. 

6. Timestep

> The entire process of doing an action and receiving/not receiving an award. (Episodes are made of timesteps.)

Make sure you fully understand the definitions (and implications) of each of these terms.

# Intro to RL part 2

## Return

Now that we know how to interact with an environment, how do we make our agent intelligent?

Our goal is to teach the agent how to choose the best action for any state it happens to be in. 

But how do we define "best"? Should the agent choose the actions that give it a reward most quickly (favoring gaining rewards in closer timesteps), or should it hold out for a bigger reward further in the future? We weigh these options against each other by defining "return" $G$:

$$G = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... = \sum_{k=0}^T \gamma^{k} R_{t+1 + k}$$

$$G_{t} = R_{t+1} + \gamma G_{t+1}$$

where $R_{t+1+k}$ is the reward at each timestep, and $\gamma$ is a discount factor (a number between 0 and 1).

This is a mathematical expression of how valuable a certain sequence of states and actions is -- it weights sooner return more heavily than later return. $\gamma$ is a hyperparameter, meaning that we can change it to make the agent care more or less about rewards further in the future.

To make sure you understand return, write a function that takes in $\gamma$ and a sequence of future rewards starting at the current timestep, and return (from the function) the return $G$. 

In [ ]:
def get_return(gamma, rewards):
  k = 0
  ret = 0
  for reward in rewards:
    ret += (gamma**k)*reward
    k+=1
  return(ret)
    
  
  

Test your function:

In [ ]:
print(get_return(0.9, [2,3,2,3,2,3,2]))
print(get_return(0.9, [0,0,0,1,0,0,0]))
print(get_return(0.9, [0,0,0,0,0,1,0]))
print(get_return(0.8, [0,0,0,0,0,1,0]))
print(get_return(0.9, [1,0,0,0,0,0,0]))
# create more tests to help answer the questions below

12.653552000000003
0.7290000000000001
0.5904900000000001
0.3276800000000001
1.0


How does changing $\gamma$ affect the return?

> making gamma smaller makes the return smaller.

How does changing the order of the rewards affect the return?

> Rewards given sooner return greater values. Greater rewards also give a greater return.

## Probability

We define $p(s', r \mid s,a)$ to be the probability of moving to state $s'$ and receiving a reward $r$ given that we are in state $s$ and take action $a$.

## Another note about environment

In the previous notebook we worked with the FrozenLake environment. In this environment, whenever we took the action "move down" (e.g.) the state changed in a deterministic way, meaning we were certain of the next state the agent would enter.

Not all environments are deterministic, however. If we set `is_slippery=True` in the FrozenLake environment, when the agent takes the action "move down" it is no longer certain that the state will change to the square below. There is a chance that the agent will end up in one of a few different states. This is a *stochastic* environment. We represent the probability of changing states as either $p(s' \mid s, a)$ (which is equal to $\sum_{r\in\mathcal{R}} p(s', r \mid s, a)$), or as $\mathcal{P}_{s,s'}^a$. ($r \in \mathcal{R}$ denotes all the possible rewards.) Both of these can be read as "the probability of moving to state $s'$ given that we are currently in state $s$ and we take action $a$".

### Even further complexity

Often, in real world examples, the agent is not able to see the complete state of the environment. While the state of the environment may currently be $s_e$, the agent may observe the state to be $s_a \neq s_e$. This is called a partially-observable environment.

Rewards are rarely certain, either. Sometimes we are not sure what reward we will get for moving into a certain state. We might get a different reward from an environment even if we take the same actions and end up in the same states.

In this project, though, we will only work with fully-observable environments and deterministic rewards.

## Expected Reward / Expected Return

Even though we don't always know exactly what reward we will receive, we can still make decisions based on what we *expect* the reward to be. It is essentially a weighted average of all the possible rewards based on how probable they are.

The expectation of a reward $R_t$ at timestep $t$ given some state $s_{t-1}$ at timestep $t-1$ and action $a_{t-1}$ at $t-1$ is written as $$\mathbb{E}[R_t \mid S_{t-1}=s, A_{t-1}=a]$$. This can be expressed as $$\sum_{r\in\mathcal{R}} r \sum_{s'\in\mathcal{S}} p(s', r \mid s, a)$$ using the probability notation from before.

**Question**: How would you express expected reward given $s$, $a$, and $s'$?

*Hint: what are you given, and what do you not know?*

> $\sum_{r\in\mathcal{R}} r *p( s', r\mid  s, a ) /  p(s' \mid s, a)$

## Policy

In order to choose an action, an agent will follow a policy function, $\pi$. We define $\pi(s,a)$ to be the probability that the agent will take action $a$ when it is in state $s$. If the agent follows a deterministic policy, $\pi(s,a) = 1$ for any given $s$ and $a$. If the agent follows a stochastic policy, then $\pi(s,a) < 1$ for any given $s$ and $a$. This means that the agent won't always take the same action in the same state.

The agent must learn a good policy in order to take actions that will lead to a high return.

## State-value and Action-value functions

There are two main ways to think about how to achieve a high return: using the state-value function or using the action-value function. Each of these functions express the expected return if the agent follows a specific policy $\pi$ and given $s$ or $s$ and $a$.

1. **State-value** function

The state-value function operates by associating each *state* with a reward. We define a function $v : S \rightarrow R$ as $$v(s) = \mathbb{E}[G_t \mid S_{t-1}=s]$$


2. **Action-value** function

The action-value function is similar to the state-value function, except the action-value function takes into account the agent taking a specific action as well as a state. We define a function $q : S, A \rightarrow R$ as $$q(s,a) = \mathbb{E}[G_t \mid S_{t-1}=s, A_{t-1}=a]$$

**Question**: How would you express the state-value function in terms of the action-value function?

> $v(s) = $




<hr>

Note: If you just get through this first part today, that's fine

<hr>

## Q Learning

Q learning is one approach to RL that focuses on the action-value function, $q$. The policy $\pi$ is implicitly defined by taking the actions that lead to the highest return, according to $q$. You will sometimes see $q$ written as $q_\pi$.

However, there's an issue. We don't know what the return is for any of the states and actions. We need to figure out the return of each state and action by allowing the agent to move around its environment and keeping track of the rewards it receives.

Create a table to store what we think is the return for each state and action in the FrozenLake environment.

(You want an entry for each state and each action. Set the dimensions of the table so that you can address it as `Q[s][a]`)

In [ ]:
Q = [[0 for a in range(4)] for s in range(16)] # edit this line
# this is a fancy way of saying "create a ? x ? matrix and initialize all the values to 0"



def printQ(Q):
  for i in range(len(Q)):
    print(Q[i])


printQ(Q)

#Q[2][3] = 2  Q[state][action]


[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


We will define our policy $\pi$ to choose the action that will lead to the highest-valued next expected state. Write a function that would choose the optimal action given the current `Q` and current state (since the values of `Q` are not yet filled in, you don't have to test this function yet):

In [ ]:
import random
def choose_optimal_a(s,Q):
    check_up = Q[s][3]
    check_down = Q[s][1]
    check_left = Q[s][0]
    check_right = Q[s][2]
    best_a = max(check_up, check_down, check_right, check_left)
    if check_up == check_down and check_left == check_right and check_up == check_right:
      a_index = random.randint(0,3)
    elif check_up==best_a:
      a_index = 3
    elif check_down==best_a:
      a_index = 1
    elif check_left==best_a:
      a_index = 0
    elif check_right==best_a:
      a_index = 2
    return a_index
choose_optimal_a(1,Q)

1

Every time the agent takes a certain action and the environment returns a certain new state and reward, we want to move the values in `Q` closer to their true values. Remember, in our case, $\pi$ is to always take the action that we think has the highest return, according to `Q`. In this way, when we update `Q` we are also updating $\pi$

To update the table, we follow the Bellman equation:

$$q_\pi(s,a) = r + \gamma \max_{a'}q(s',a') $$

Intuitively, this function is looking back at the expected return for $s$ and $a$ given that we have now taken that we have taken a step forward. It is like using hindsight to make better predictions in the future.

However, since we are continuously updating $q$ (and therefore $\pi$), we don't want to completely override the previous values of `Q`, as that would not make for a stable learning process. We want to ajust them slightly. 

Write some code that updates your table according to the Bellman equation (it should take an action, receive a reward, and update the appropriate entry in `Q`):

In [ ]:
import gym
env = gym.make("FrozenLake-v0", is_slippery=True)
s=2
a=0
Q[s][a]=0
r=2
check=[]
for i in range(4):
  check+=[Q[s][i]]
gamma=.1
Q[s][a] = r + gamma*(max(check))
print(Q[s][a])

2.0


In [ ]:
from tqdm import tqdm
import time

for i in tqdm(range(10)):
  time.sleep(.5)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


We end up with the following training loop. Complete the code inside the loop.

In [ ]:
env = gym.make("FrozenLake-v0", is_slippery=True)


# don't forget to reset Q
Q = [[0 for a in range(4)] for s in range(16)] 

epsilon = 0.9


num_epochs = 100000

for i in tqdm(range(num_epochs)):
  s = env.reset()
  done = False
  while not done:
    somethin = random.random()
    if somethin < epsilon:
      a = random.randint(0,3)
    else:
      a= choose_optimal_a(s,Q)
    # take action
    old_state = s
    s, reward, done, info = env.step(a)
    
    if s != 15 and done == True:
      reward -=1

    
    # update table
    
    check=[]
    for i in range(4):
      check+=[Q[s][i]]
    gamma=.9
    Q[old_state][a] = (0.9*Q[old_state][a]+ 0.1*(reward + gamma*(max(check))))
    epsilon *= 0.99896


    
env.render()   
printQ(Q)


100%|██████████| 100000/100000 [01:07<00:00, 1488.28it/s]

  (Down)
SFFF
FHFH
FFFH
HFFG
[0.012128197827990248, -0.15904662757905633, -0.1565571818883713, -0.15969154983272316]
[-0.27186812117624354, -0.2592871484413749, -0.3581906115736314, 0.018600776299999115]
[-0.1989618631290543, -0.2605196705931636, -0.20620202816598543, -0.017675346470209925]
[-0.3178637620565129, -0.39762966893698715, -0.324627482720111, -0.03729807723720214]
[0.01994916367230312, -0.28653949828186936, -0.27462248457147265, -0.32253965553531855]
[0, 0, 0, 0]
[-0.7694443886979748, -0.7736455264247891, -0.21205622133195565, -0.7854412530278388]
[0, 0, 0, 0]
[-0.3234120918003049, -0.44404843231995517, -0.2617772515004455, 0.04891814049225932]
[-0.2585903648352459, 0.17354469986448123, -0.1550778336881107, -0.26875492967881753]
[0.2930486344038311, -0.2888312508781502, -0.32489638939449017, -0.30834247258373165]
[0, 0, 0, 0]
[0, 0, 0, 0]
[-0.20517039941985232, -0.1304984312045525, 0.2719072195175513, -0.07859277986364914]
[0.11276832939675449, 0.7427208533012898, 0.09760507

In [ ]:
env = gym.make("FrozenLake-v0", is_slippery=True)
s = env.reset()
success = 0


for k in range(20000):
  done = False
  s= env.reset()
  while done != True:
    new_a = choose_optimal_a(s, Q)
    s, r, done, info = env.step(new_a)
  if s ==15:
    success +=1
print(success)
print(success/20000)

14816
0.7408


Now watch your agent move quickly across the frozen lake without falling into any holes! Write a function that uses the Q table you trained to guide and render the agent moving through the environment. You can adapt your code from the last question of the previous notebook.

In [ ]:
import random

env = gym.make("FrozenLake-v0", is_slippery=True)

decimal_num=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

# don't forget to reset Q
Q = [[0 for a in range(4)] for s in range(16)] 

for i in range(len(decimal_num)):
  success2=0
  epsilon=decimal_num[i]
  for j in range(len(decimal_num)):
    gamma=decimal_num[j]
    for k in range(len(decimal_num)):
      learning_r=decimal_num[k]
      l_rate2=1-learning_r


  num_epochs = 100000

  for i in tqdm(range(num_epochs)):
    s = env.reset()
    done = False
    while not done:
      somethin = random.random()
      if somethin < epsilon:
        a = random.randint(0,3)
      else:
        a= choose_optimal_a(s,Q)
      # take action
      old_state = s
      s, reward, done, info = env.step(a)

      if s != 15 and done == True:
        reward -=1


      # update table

      check=[]
      for i in range(4):
        check+=[Q[s][i]]
      Q[old_state][a] = (l_rate2*Q[old_state][a]+ learning_r*(reward + gamma*(max(check))))
      epsilon *= 0.999999

  env = gym.make("FrozenLake-v0", is_slippery=True)
  s = env.reset()
  success = 0


  for k in range(20000):
    done = False
    s= env.reset()
    while done != True:
      new_a = choose_optimal_a(s, Q)
      s, r, done, info = env.step(new_a)
    if s ==15:
      success +=1
    success3=success/20000
    if success3> success2:
      success2=success3
      learning=learning_r
      gamma2=gamma
      epsilon2=epsilon
print("learning rate:",learning)
print("gamma:",gamma2)
print("epsilon:",epsilon2)
      
  


100%|██████████| 100000/100000 [00:16<00:00, 6186.89it/s]


learning rate: 0.9
gamma: 0.9
epsilon: 0.3519951136791793


#### Additional Resources

* [https://towardsdatascience.com/introduction-to-q-learning-88d1c4f2b49c](https://towardsdatascience.com/introduction-to-q-learning-88d1c4f2b49c)